In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

In [4]:
from wordvecs import WordVectors

wordvectors = WordVectors(fname="../enwiki-20141208-pages-articles-multistream-links-output4.bin", negvectors=True)

from sentiment_sents import Sentiment

# just load the sentences from the CNN system
sentiment = Sentiment("prevwork/CNN_sentence/")

In [5]:
len(wordvectors.vectors)

6091842

In [6]:
l_in = InputLayer((3,))

In [7]:
W = np.arange(3*5).reshape((3,5)).astype('float32')

In [8]:
l1 = EmbeddingLayer(l_in, input_size=3, output_size=5, W=W)

In [9]:
x = T.imatrix()

In [10]:
output = get_output(l1, x)

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "


In [11]:
f = theano.function([x], output)

In [12]:
x_test = np.array([[0,2],[1,2]]).astype('int32')

In [13]:
f(x_test)

array([[[  0.,   1.,   2.,   3.,   4.],
        [ 10.,  11.,  12.,  13.,  14.]],

       [[  5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.]]], dtype=float32)

Going to attempt to make the simple linear reg working, just to experiment with theano

In [14]:
np.random.randn(10)

array([ 0.3381165 , -1.04912337, -0.0079118 ,  0.29713418, -0.39379286,
       -0.49857677,  0.24749438, -1.52724946, -0.91314897, -0.43407606])

In [15]:
nfeats = 10


x = T.matrix('x')
y = T.vector('y')
w = theano.shared(np.random.randn(nfeats),name='w')
b = theano.shared(0.0,name='b')

In [16]:
p_1 = 1 / (1 + T.exp(-T.dot(x,w) - b))
prediction = p_1 > 0.5
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1)
cost = xent.mean() + .01 * (w**2).sum()
gw, gb = T.grad(cost, [w,b])

In [17]:
w.get_value().shape

(10,)

In [18]:
train = theano.function(inputs=[x,y],outputs=[prediction,xent], updates=((w, w-0.1*gw),(b,b - 0.1*gb)))
predict = theano.function(inputs=[x], outputs=prediction)

In [19]:
D = (np.random.randn(400, nfeats), np.random.randint(size=400, low=0, high=2))

In [20]:
theano.printing.debugprint(train)

Elemwise{Composite{GT(scalar_sigmoid(i0), i1)}} [@A] ''   15
 |Elemwise{neg,no_inplace} [@B] ''   13
 | |Elemwise{Composite{((-i0) - i1)}}[(0, 0)] [@C] ''   10
 |   |CGemv{inplace} [@D] ''   8
 |   | |Alloc [@E] ''   5
 |   | | |TensorConstant{0.0} [@F]
 |   | | |Shape_i{0} [@G] ''   1
 |   | |   |x [@H]
 |   | |TensorConstant{1.0} [@I]
 |   | |x [@H]
 |   | |w [@J]
 |   | |TensorConstant{0.0} [@F]
 |   |InplaceDimShuffle{x} [@K] ''   0
 |     |b [@L]
 |TensorConstant{(1,) of 0.5} [@M]
Elemwise{Composite{((i0 * scalar_softplus(i1)) - (i2 * i3 * scalar_softplus(i4)))}}[(0, 1)] [@N] ''   17
 |y [@O]
 |Elemwise{Composite{((-i0) - i1)}}[(0, 0)] [@C] ''   10
 |TensorConstant{(1,) of -1.0} [@P]
 |Elemwise{sub,no_inplace} [@Q] ''   4
 | |TensorConstant{(1,) of 1.0} [@R]
 | |y [@O]
 |Elemwise{neg,no_inplace} [@B] ''   13
CGemv{inplace} [@S] ''   19
 |w [@J]
 |TensorConstant{-0.1} [@T]
 |InplaceDimShuffle{1,0} [@U] 'x.T'   2
 | |x [@H]
 |Elemwise{Composite{((i0 * i1 * i2 * i3) + ((scalar_sigmoi

In [21]:
for i in xrange(10000):
    pred, err = train(D[0], D[1])

In [22]:
err.mean()

0.68227781271962551

In [23]:
D[0][1]

array([ 1.44959463,  0.46512157, -0.32916993, -0.71065438,  0.35574081,
        0.42292786, -0.38971277,  0.39731118, -0.40333132, -2.43409073])

In [24]:
w.get_value(), b.get_value()

(array([ 0.06392266,  0.0166076 ,  0.10579282,  0.11662214,  0.06546423,
         0.03236646,  0.03197849, -0.09175565,  0.00399894, -0.17068669]),
 array(0.02220269693379262))

In [25]:
predict([D[0][1]])

array([1], dtype=int8)

In [26]:
D[1][1]

0

going to see if we can make a few layers of a cnn given that went through the example of log reg from the tutorial

In [27]:
def make_context(sentence):
    ret = []
    for w in sentence.lower().split():
        ret.append(wordvectors[w])
    return ret

In [204]:
class SentimentExp(object):
    
    def __init__(self, train_X, train_Y, wordvecs=wordvectors):
        self.train_X = train_X
        self.train_Y = theano.shared(np.array(train_Y, 'int32'))
        self.wordvecs = wordvecs
        
        self.input_size = 10
        self.batch_size = 10
        
        self.learning_rate = .01
        self.momentum = .9
        
        self.train_X_rep = theano.shared(lasagne.utils.floatX([[self.getRep(x)] for x in self.train_X]))
                
        self._setup()
        
    def getRep(self, sent):
        ret = []
        for i in xrange(self.input_size):
            if i < len(sent):
                ret.append(self.wordvecs[sent[i]])
            else:
                ret.append(np.zeros(self.wordvecs.vector_size))
        return np.matrix(ret).reshape((1, self.input_size, self.wordvecs.vector_size))
    
    def _setup(self):
        self.y_batch = T.ivectors('y')
        self.x_batch = T.tensor4('x')  # have to match the dimention of the input layer
        
        self.batch_index = T.iscalar('batch_index')
        self.batch_slice = slice(self.batch_index * self.batch_size, (self.batch_index + 1) * self.batch_size)
        
        self.input_l = InputLayer(shape=(self.batch_size, 1, self.input_size, self.wordvecs.vector_size), name='x')
        self.conv1_l = lasagne.layers.Conv2DLayer(
            self.input_l,
            num_filters=32, 
            filter_size=5,
            name='conv1'
        )
        #self.conv2_l = lasagne.layers.Conv2DLayer(
        #   self.conv1_l,
        #    num_filters=32,
        #    filter_size=5,
        #)
        self.output_l = lasagne.layers.DenseLayer(self.conv1_l, num_units=1, name='dens2')
        
        self.output_v = lasagne.layers.get_output(self.output_l, self.x_batch)
        self.loss_v = lasagne.objectives.binary_crossentropy(self.output_v, self.y_batch).mean()
        
        self.output_test = lasagne.layers.get_output(self.output_l, self.x_batch, deterministic=True)
        self.loss_eval = lasagne.objectives.binary_crossentropy(self.output_test, self.y_batch)
        
        self.all_params = lasagne.layers.get_all_params(self.output_l)
        self.updates = lasagne.updates.nesterov_momentum(self.loss_v, self.all_params, 
                                                         self.learning_rate, self.momentum)
        
        self.prediction = T.argmax(self.output_v)
        self.accuract = T.mean(T.eq(self.prediction, self.y_batch), dtype=theano.config.floatX)
        
        self.iter_train = theano.function(
            [self.batch_index], self.loss_v,
            updates=self.updates,
            givens={
                self.x_batch: self.train_X_rep[self.batch_slice],
                self.y_batch: self.train_Y[self.batch_slice]
            },
            mode='DebugMode'
        )
        
        self.test_valid = theano.function(
            [self.x_batch, self.y_batch], [self.loss_eval, self.prediction, self.accuract],
        )
            
    def train(self):
        pass
            
experiment = SentimentExp(sentiment.train_X, sentiment.train_Y)

In [187]:
theano.config.optimizer = 'None'  # disable the optimizer so that we can maybe get more debugging information
theano.config.exception_verbosity = 'high'

In [195]:
experiment.input_l.shape, experiment.train_X_rep[experiment.batch_slice].eval({experiment.batch_index:0}).shape

((10, 1, 10, 100), (10, 1, 10, 100))

In [196]:
experiment.train_Y[experiment.batch_slice].eval({experiment.batch_index:0}).shape

(10,)

In [205]:
experiment.iter_train(0)

InvalidValueError: InvalidValueError
        type(variable) = TensorType(float64, matrix)
        variable       = Elemwise{log,no_inplace}.0
        type(value)    = <type 'numpy.ndarray'>
        dtype(value)   = float64
        shape(value)   = (10, 1)
        value          = [[       -inf]
 [-2.62899249]
 [       -inf]
 [       -inf]
 [       -inf]
 [       -inf]
 [       -
        min(value)     = -inf
        max(value)     = -2.03492905143
        isfinite       = False
        client_node    = None
        hint           = perform output
        specific_hint  = non-finite elements not allowed
        context        = ...
  Elemwise{log,no_inplace} [@A] ''   
   |Elemwise{mul,no_inplace} [@B] ''   
     |DimShuffle{x,x} [@C] ''   
     | |TensorConstant{0.5} [@D]
     |Elemwise{add,no_inplace} [@E] ''   
       |Elemwise{add,no_inplace} [@F] ''   
       | |dot [@G] ''   
       | | |Flatten{2} [@H] ''   
       | | | |Elemwise{mul,no_inplace} [@I] ''   
       | | |   |DimShu

In [31]:
from helpers import *

In [46]:
batch_index = T.iscalar('batch_index')
X_batch = T.matrix('x.input')
y_batch = T.ivector('y')
slice(batch_index * 100, (batch_index + 1) * 100)

slice(Elemwise{mul,no_inplace}.0, Elemwise{mul,no_inplace}.0, None)

In [61]:
experiment.

Elemwise{true_div,no_inplace}.0

In [40]:
experiment.input.name

In [49]:
l_in = lasagne.layers.InputLayer(shape=(10,1,10,10))
l_conv1 = lasagne.layers.Conv2DLayer(l_in, num_filters=32, filter_size=(5,5))
l_out = lasagne.layers.DenseLayer(l_conv1, num_units=1)

In [50]:
lasagne.layers.get_output(l_out)

Elemwise{mul,no_inplace}.0

In [79]:
sentiment.train_X[0]

'provide[s] nail-biting suspense and credible characters without relying on technology-of-the-moment technique or pretentious dialogue . \n'

In [81]:
theano.

AttributeError: 'module' object has no attribute 'warn'

In [84]:
X_batch

x.input

In [96]:
theano.shared(np.array(sentiment.train_Y, 'int32'))

<TensorType(int32, vector)>

In [99]:
T.ivector()

<TensorType(int32, vector)>

TypeError: 'type' object has no attribute '__getitem__'